In [ ]:
import pathlib, os, shutil , random

base_dir= pathlib.Path('./')
val_dir = base_dir/'val'
os.makedirs(val_dir)
train_dir = base_dir/'train'



In [ ]:
for category in ('neg', 'pos'):
    os.makedirs(val_dir/category)
    files = os.listdir(train_dir/category)
    random.shuffle(files)
    files=list(files)
    num_val_samples = int (0.2 *len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir/category/fname,val_dir/category/fname)
    


In [5]:
from tensorflow import keras 
batch_size = 32
train_ds = keras.utils.text_dataset_from_directory('./train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory('./val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory('./test', batch_size=batch_size)

Found 20000 files belonging to 2 classes.
Metal device set to: Apple M2 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-07-28 14:25:15.470179: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-07-28 14:25:15.470673: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [6]:
import tensorflow as tf

In [7]:
text_vectorization = tf.keras.layers.TextVectorization(max_tokens=20000, output_mode='multi_hot')
text_only_train_ds = train_ds.map(lambda x,y:x)
text_vectorization.adapt(text_only_train_ds)

2023-07-28 14:25:20.607120: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-07-28 14:25:20.663069: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [8]:
binary_lgram_train_ds = train_ds.map(lambda x , y:((text_vectorization(x), y)), num_parallel_calls=6)
binary_lgram_val_ds = val_ds.map(lambda x , y:((text_vectorization(x), y)), num_parallel_calls=6)
binary_lgram_test_ds = test_ds.map(lambda x , y:((text_vectorization(x), y)), num_parallel_calls=6)

In [9]:
def get_model(max_tokens=20000, hidden_dim = 16):
    inputs = tf.keras.Input(shape=(max_tokens,))
    x = tf.keras.layers.Dense(hidden_dim, activation='relu')(inputs)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
    model = tf.keras.Model(inputs,outputs)
    model.compile(optimizer='rmsprop', loss=('binary_crossentropy'), metrics=['accuracy'])
    return model

In [10]:
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(binary_lgram_train_ds,validation_data=binary_lgram_val_ds,epochs=10)

Epoch 1/10


2023-07-23 18:50:25.111415: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


623/625 [============================>.] - ETA: 0s - loss: 0.3851 - accuracy: 0.8436

2023-07-23 18:50:32.025619: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 8s 10ms/step - loss: 0.3852 - accuracy: 0.8436 - val_loss: 0.2838 - val_accuracy: 0.8892
Epoch 2/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2591 - accuracy: 0.9051 - val_loss: 0.2820 - val_accuracy: 0.8874
Epoch 3/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2284 - accuracy: 0.9207 - val_loss: 0.2698 - val_accuracy: 0.8926
Epoch 4/10
625/625 [==============================] - 6s 10ms/step - loss: 0.2059 - accuracy: 0.9320 - val_loss: 0.2777 - val_accuracy: 0.8954
Epoch 5/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1956 - accuracy: 0.9355 - val_loss: 0.2847 - val_accuracy: 0.8926
Epoch 6/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1840 - accuracy: 0.9427 - val_loss: 0.2953 - val_accuracy: 0.8926
Epoch 7/10
625/625 [==============================] - 6s 10ms/step - loss: 0.1876 - accuracy: 0.9431 - val_loss: 0.3010 - val_accuracy: 0.8938
Epoch 8/10

# sequence 

In [24]:
max_lenght = 600
max_tokens = 20000
text_vectorization = tf.keras.layers.TextVectorization(max_tokens=max_tokens, output_mode='int', output_sequence_length= max_lenght)
text_vectorization.adapt(text_only_train_ds)

2023-07-28 14:38:36.106630: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


In [26]:
int_lgram_train_ds = train_ds.map(lambda x , y:((text_vectorization(x), y)), num_parallel_calls=6)
int_lgram_val_ds = val_ds.map(lambda x , y:((text_vectorization(x), y)), num_parallel_calls=6)
int_lgram_test_ds = test_ds.map(lambda x , y:((text_vectorization(x), y)), num_parallel_calls=6)

In [31]:
inputs = tf.keras.Input(shape=(600,),dtype='int64')
embedded = tf.one_hot(inputs, depth=max_tokens)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)) (embedded)
x = tf.keras.layers.Dropout(0.5) (x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs,outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 600)]             0         
                                                                 
 tf.one_hot_2 (TFOpLambda)   (None, 600, 20000)        0         
                                                                 
 bidirectional_2 (Bidirectio  (None, 64)               5128448   
 nal)                                                            
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_4 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,128,513
Trainable params: 5,128,513
Non-trainable params: 0
_________________________________________________

### this model trains slowly. using one-hot encoding ( as a simplest thing we could do) was not a great idea.

In [ ]:
model.fit(int_lgram_train_ds,validation_data=int_lgram_val_ds, epochs=10)


## Model uses Embedding layer

In [34]:
inputs = tf.keras.Input(shape=(None,), dtype= 'int64')
embedded = tf.keras.layers.Embedding(input_dim=max_tokens,output_dim=256)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(embedded)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         5120000   
                                                                 
 bidirectional_3 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [35]:
model.fit(int_lgram_train_ds, validation_data=int_lgram_val_ds)

2023-07-28 15:35:52.069124: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:35:52.527522: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:35:52.547801: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:35:55.727271: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:35:55.756046: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - ETA: 0s - loss: 0.4714 - accuracy: 0.7894

2023-07-28 15:39:54.065427: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:39:54.156440: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:39:54.167741: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 274s 420ms/step - loss: 0.4714 - accuracy: 0.7894 - val_loss: 0.3907 - val_accuracy: 0.8296


#### embedding layer with masking enabled

In [37]:
inputs = tf.keras.Input(shape=(None,), dtype= 'int64')
embedded = tf.keras.layers.Embedding(input_dim=max_tokens,output_dim=256, mask_zero=True)(inputs)
x = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32))(embedded)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 256)         5120000   
                                                                 
 bidirectional_5 (Bidirectio  (None, 64)               73984     
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 64)                0         
                                                                 
 dense_7 (Dense)             (None, 1)                 65        
                                                                 
Total params: 5,194,049
Trainable params: 5,194,049
Non-trainable params: 0
_________________________________________________

In [38]:
model.fit(int_lgram_train_ds, validation_data=int_lgram_val_ds)

2023-07-28 15:44:21.430201: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:44:21.898834: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_40/output/_23'
2023-07-28 15:44:21.904940: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:44:22.064509: I tensorflow/co

625/625 [==============================] - ETA: 0s - loss: 0.6944 - accuracy: 0.5048

2023-07-28 15:48:03.865195: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:48:04.077155: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2023-07-28 15:48:04.148608: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


625/625 [==============================] - 261s 400ms/step - loss: 0.6944 - accuracy: 0.5048 - val_loss: 0.6929 - val_accuracy: 0.5002


### you can use pretrained word embdeddings ( word2ve/ Glove)